In [ ]:
def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
    if self.use_norm:
        # Normalization from Non-stationary Transformer
        means = x_enc.mean(1, keepdim=True).detach()
        x_enc = x_enc - means
        stdev = torch.sqrt(torch.var(x_enc, dim=1, keepdim=True, unbiased=False) + 1e-5)
        x_enc /= stdev

    _, _, N = x_enc.shape

    en_embed, n_vars = self.en_embedding(x_enc[:, :, -1].unsqueeze(-1).permute(0, 2, 1)) # pass target only for embedding
    ex_embed = self.ex_embedding(x_enc[:, :, :-1], x_mark_enc) # pass other variables for embedding, with time information

    enc_out = self.encoder(en_embed, ex_embed)
    enc_out = torch.reshape(
        enc_out, (-1, n_vars, enc_out.shape[-2], enc_out.shape[-1]))
    # z: [bs x nvars x d_model x patch_num]
    enc_out = enc_out.permute(0, 1, 3, 2)

    dec_out = self.head(enc_out)  # z: [bs x nvars x target_window]
    dec_out = dec_out.permute(0, 2, 1)

    if self.use_norm:
        # De-Normalization from Non-stationary Transformer
        dec_out = dec_out * (stdev[:, 0, -1:].unsqueeze(1).repeat(1, self.pred_len, 1))
        dec_out = dec_out + (means[:, 0, -1:].unsqueeze(1).repeat(1, self.pred_len, 1))

    return dec_out
